In [5]:
import numpy as np
import pulp

In [2]:
def get_class_center(Xs, Ys):
    class_center = []
    classes = np.unique(Ys)

    for c in classes:
        mask = Ys == c
        Xi = Xs[mask.flatten()]
        hc = np.mean(Xi, axis=0)
        class_center.append(hc)

    return np.asarray(class_center)

In [3]:
def get_distance_matrix(Xt, class_center):
    nt = Xt.shape[0]
    C = class_center.shape[0]

    D = np.zeros((C, nt))

    for i, hc in enumerate(class_center):
        for j, xt in enumerate(Xt):
            d = np.linalg.norm(xt - hc)
            D[i][j] = d

    return D

In [8]:
def solve_LP(C, nt, Dcj):

    Dcj = abs(Dcj)
    model = pulp.LpProblem('Cost minimizing problem', pulp.LpMinimize)
    Mcj = pulp.LpVariable.dicts('Probabality',
                                ((i, j) for i in range(C) for j in range(nt)),
                                lowBound=0,
                                upBound=1,
                                cat='continous')
    # Objective function
    model += (pulp.lpSum(Dcj[(i, j)] * Mcj[(i, j)] for i in range(C)
                         for j in range(nt)))

    # Constarints
    for j in range(nt):
        model += pulp.lpSum([Mcj[(i, j)] for i in range(C)]) == 1

    for i in range(C):
        model += pulp.lpSum([Mcj[(i, j)] for j in range(nt)]) >= 1

    # Solve the problem
    model.solve()
    pulp.LpStatus[model.status]
    output = [[Mcj[i, j].varValue for i in range(C)] for j in range(nt)]

    return np.array(output)

In [9]:
def intra_domain_programming(Xs, Ys, Xt, Yt):
    C = len(np.unique(Ys))
    nt = len(Yt)

    class_center = get_class_center(Xs, Ys)
    D = get_distance_matrix(Xt, class_center)
    Mcj = solve_LP(C, nt, D)

    y_pred = np.argmax(Mcj, axis=1)
    acc = np.mean(y_pred == Yt.flatten())

    return y_pred, acc